## Build MRCNN Model
Pass data through MRCNN and then FCN and investigte output values from FCN
- First we generate the heatmaps, and also visually cehck them. 
- the we pass the heatmaps to the routine that prodcues the scores 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
 
import os, sys, math, io, time, gc, argparse, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')
import mrcnn.model_mrcnn  as mrcnn_modellib
import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize
import mrcnn.new_shapes   as shapes
from datetime import datetime   
from mrcnn.utils        import command_line_parser, Paths
from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
from mrcnn.prep_notebook import mrcnn_coco_train, prep_coco_dataset

pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

####  Pass input parameters to argparse

# args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
# input_parms = "--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
# input_parms +=" --model     /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 "
##------------------------------------------------------------------------------------
## Parse command line arguments
##------------------------------------------------------------------------------------
parser = command_line_parser()
input_parms = "--epochs 2 --steps_in_epoch 32  --last_epoch 0 --batch_size 1 --lr 0.00001 --val_steps 8 " 
input_parms +="--mrcnn_logs_dir train_mrcnn_coco "
input_parms +="--fcn_logs_dir   train_fcn8_coco "
input_parms +="--mrcnn_model    last "
input_parms +="--fcn_model      init "
input_parms +="--opt            adagrad "
input_parms +="--fcn_arch       fcn8 " 
input_parms +="--fcn_layers     all " 
input_parms +="--sysout        screen "
input_parms +="--new_log_folder    "
# input_parms +="--fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

args = parser.parse_args(input_parms.split())
# args = parser.parse_args()

##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
# debug = False
if args.sysout == 'FILE':
    sys.stdout = io.StringIO()

# print("    Dataset            : ", args.dataset)
# print("    Logs               : ", args.logs)
# print("    Limit              : ", args.limit)
print("    MRCNN Model        : ", args.mrcnn_model)
print("    FCN Model          : ", args.fcn_model)
print("    MRCNN Log Dir      : ", args.mrcnn_logs_dir)
print("    FCN Log Dir        : ", args.fcn_logs_dir)
print("    FCN Arch           : ", args.fcn_arch)
print("    FCN Log Dir        : ", args.fcn_layers)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)
print("    Optimizer          : ", args.opt)
print("    sysout             : ", args.sysout)
# print("    OS Platform        : ", syst)

##------------------------------------------------------------------------------------
## setup project directories
##   ROOT_DIR         : Root directory of the project 
##   MODEL_DIR        : Directory to save logs and trained model
##   COCO_MODEL_PATH  : Path to COCO trained weights
##---------------------------------------------------------------------------------
paths = Paths(fcn_training_folder = args.fcn_logs_dir, mrcnn_training_folder = args.mrcnn_logs_dir)
paths.display()

##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------                          
mrcnn_config                    = CocoConfig()
mrcnn_config.NAME               = 'mrcnn'              
mrcnn_config.TRAINING_PATH      = paths.MRCNN_TRAINING_PATH
mrcnn_config.COCO_DATASET_PATH  = paths.COCO_DATASET_PATH 
mrcnn_config.COCO_MODEL_PATH    = paths.COCO_MODEL_PATH   
mrcnn_config.RESNET_MODEL_PATH  = paths.RESNET_MODEL_PATH 
mrcnn_config.VGG16_MODEL_PATH   = paths.VGG16_MODEL_PATH  
mrcnn_config.COCO_CLASSES       = None 
mrcnn_config.DETECTION_PER_CLASS = 200
mrcnn_config.HEATMAP_SCALE_FACTOR = 4
mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE

mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)

# mrcnn_config.WEIGHT_DECAY       = 2.0e-4
# mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
# mrcnn_config.REDUCE_LR_FACTOR   = 0.5
# mrcnn_config.REDUCE_LR_COOLDOWN = 30
# mrcnn_config.REDUCE_LR_PATIENCE = 40
# mrcnn_config.EARLY_STOP_PATIENCE= 80
# mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
# mrcnn_config.MIN_LR             = 1.0e-10
# mrcnn_config.OPTIMIZER          = args.opt.upper()
# mrcnn_model.config.OPTIMIZER    = 'ADAGRAD'
mrcnn_config.NEW_LOG_FOLDER       = False
mrcnn_config.SYSOUT               = args.sysout
mrcnn_config.display() 


## Defined training datasets

In [ ]:
##------------------------------------------------------------------------------------
## Build & Load Training and Validation datasets
##------------------------------------------------------------------------------------
# dataset_train = prep_coco_dataset(["train",  "val35k"], mrcnn_config, generator = False)
# dataset_val   = prep_coco_dataset(["minival"]         , mrcnn_config, generator = False)
# from mrcnn.prep_notebook import coco_dataset
# dataset_train = coco_dataset(["val35k"], mrcnn_config)
# dataset_val   = coco_dataset(["minival"], mrcnn_config)
from mrcnn.dataset      import Dataset 
from pycocotools.coco import COCO
import re

In [ ]:
HEATMAP_PATH = os.path.join(paths.DIR_DATASET,'coco2014_heatmaps')
print(HEATMAP_PATH)
from mrcnn.heatmap import HeatmapDataset
dataset = HeatmapDataset()

In [ ]:
dataset.load_heatmap(mrcnn_config.COCO_DATASET_PATH, HEATMAP_PATH, 'minival')
# def load_heatmap(self, dataset_dir, heatmap_dataset_dir, subset, class_ids=None,class_map=None, return_coco=False):
dataset.prepare()

#### simulate `load_heatmap()`

In [ ]:
dataset_dir =mrcnn_config.COCO_DATASET_PATH
heatmap_dataset_dir = HEATMAP_PATH
subset = 'minival'
class_ids=None
class_map=None
return_coco=False

In [ ]:
image_dir = os.path.join(dataset_dir, "train2014" if subset == "train" else "val2014")
heatmap_dir = os.path.join(heatmap_dataset_dir, "train2014" if subset == "train" else "val2014")
#       image_dir = os.path.join(dataset_dir, "train2017" if subset == "train" lse "val2017")
print(image_dir,'\n', heatmap_dir)
 
# Create COCO object
json_path_dict = {
    "train"  :  "annotations/instances_train2014.json",
    "val"    :  "annotations/instances_val2014.json",
    "minival":  "annotations/instances_minival2014.json",
    "val35k" :  "annotations/instances_valminusminival2014.json",
    "test"   :  "annotations/image_info_test2014.json"
}
print('subset: ', subset, 'json_path_dir: ', json_path_dict[subset])
coco = COCO(os.path.join(dataset_dir, json_path_dict[subset]))

In [ ]:
# Load all classes or a subset?
if not class_ids:
    # All classes
    class_ids = sorted(coco.getCatIds())
print(' ClassIds     :', class_ids)

In [ ]:
##--------------------------------------------------------------
## Get image ids - using COCO
##--------------------------------------------------------------
#All images or a subset?
if class_ids:
    print(' Subset of classes')
    image_ids = []
    for id in class_ids:
        image_ids.extend(list(coco.getImgIds(catIds=[id])))
    # Remove duplicates
    image_ids = list(set(image_ids))
else:
    # All images
    class_ids = sorted(coco.getCatIds())
    print(' All classes')    
    image_ids = list(coco.imgs.keys())
    
print(' ClassIds     : ', len(class_ids))
print(' Image ids    : ', len(image_ids))

In [ ]:
# # Add classes to dataset.class_info structure
for i in class_ids:
    dataset.add_class("coco", i, coco.loadCats(i)[0]["name"])

In [ ]:
len(dataset.class_info)
# image_ids[:20]

In [ ]:
# # print(' ClassIds     :', class_ids)
# # Add images to dataset.image_info structure
dataset.image_info = []
heatmap_notfound=  heatmap_found = 0
print(heatmap_notfound, heatmap_found)
for i in image_ids:
    print('image id: ',i)
    heatmap_filename = 'hm_{:012d}.npz'.format(i)
    heatmap_path = os.path.join(heatmap_dir, heatmap_filename) 
    
    ## Only load image_info data structure for images where the corrsponding 
    ## heatmap .npz file exist
    if not os.path.isfile(heatmap_path):
        print('file not found:::',heatmap_filename)
        heatmap_notfound += 1
    else:
        dataset.add_image(
            "coco", image_id=i,
            path=os.path.join(image_dir, coco.imgs[i]['file_name']),
            width=coco.imgs[i]["width"],
            height=coco.imgs[i]["height"],
            heatmap_path=heatmap_path
          )
        heatmap_found += 1
        # annotations=coco.loadAnns(coco.getAnnIds(imgIds=[i], catIds=class_ids, iscrowd=None)))
        
        
print(' Images ids :', len(image_ids))
print('    Corresponding heatmap found     :' , heatmap_found)
print('    Corresponding heatmap not found :' , heatmap_notfound)
print(' Total      :', heatmap_found + heatmap_notfound)

In [ ]:
print(len(dataset.image_ids))
print(len(dataset.image_info))
print(dataset.image_info[0])
# print(dataset.image_info[5000])

In [ ]:
##--------------------------------------------------------------
## Get image ids - using walk on HEATMAP_PATH
##--------------------------------------------------------------
print(' image dir        : ', image_dir) 
print(' json_path_dir    : ', os.path.join(dataset_dir, json_path_dict[subset]))
regex = re.compile(".*/\w+(\d{12})\.jpg")


image_ids = [] 
heatmap_files = next(os.walk(heatmap_dir))[2]
print('heat ap dir :' , heatmap_dir)

In [ ]:
for hm_file in heatmap_files:
    print(' Processing file: ', hm_file)
    heatmap_path=os.path.join(heatmap_dir, hm_file) 
    i = int(os.path.splitext(hm_file.lstrip('hm_'))[0])
    loaddata = np.load(heatmap_path)
    print(loaddata['coco_info'])
    coco_id = loaddata['coco_info'][0]
    coco_filename = loaddata['coco_info'][1]
    input_image_meta = loaddata['input_image_meta']
    loaddata.close()
    dataset.add_image(
        "coco", 
        image_id=i,
        path=os.path.join(image_dir, coco.imgs[i]['file_name']),
        width=coco.imgs[i]["width"],
        height=coco.imgs[i]["height"],
        heatmap_path=os.path.join(heatmap_dir, 'hm_{:012d}'.format(i)) 
      )    
#     print(input_filename, type(input_filename), len(input_filename))
#     coco_filename = input_filename.replace('\\' , "/")
#     print(coco_filename)
#     regex_match  = regex.match(input_filename)            
#     # Add images to dataset.image_info structure
#     if regex_match:
#         coco_id = int(regex_match.group(1))
#     print(i, input_image_meta[:8],' ', input_filename, ' coco_id : ',coco_id)

#     self.add_image(
#         "coco", 
#         image_id=i,
#         path = input_filename,
#         height=input_image_meta[1],
#         width= input_image_meta[2],
#         heatmap_path=heatmap_path
#       )
#     image_ids.append(i)
#         # annotations=coco.loadAnns(coco.getAnnIds(imgIds=[i], catIds=class_ids, iscrowd=None)))
# print(' number of images : ', len(image_ids))

#### Define data generator

In [ ]:
from mrcnn.datagen_fcn import fcn_data_generator, fcn_data_gen_simulate
##--------------------------------------------------------------------------------
## Data generators
##--------------------------------------------------------------------------------
generator = fcn_data_generator(dataset, mrcnn_config, shuffle=True,
                                 batch_size=mrcnn_config.BATCH_SIZE)
# val_generator   = data_generator(dataset_val, mrcnn_model.config, shuffle=True,
#                                  batch_size=mrcnn_config.BATCH_SIZE,
#                                  augment=False)

In [ ]:
train_batch_x, train_batch_y = next(generator)

for i in train_batch_x:
    print(type(i), i.shape, i.dtype)
for i in train_batch_y:
    print(type(i), i.shape)
print(train_batch_y)  

In [ ]:
# imgmeta_idx = mrcnn_model.keras_model.input_names.index('input_image_meta')
from mrcnn.visualize import plot_2d_heatmap_compare
import mrcnn.utils as utils
# def plot_2d_heatmap_compare( Z1, Z2, boxes, image_idx, class_ids,  size = None, 
#                                  num_bboxes = 0, class_names=None, scale = 1,
#                                  title = '2D Comparison between 2d heatmaps w/ bboxes'):
train_batch_x, train_batch_y = fcn_data_gen_simulate(dataset, mrcnn_config, [210])
img_meta    = train_batch_x[1]
class_names = dataset.class_names
print(img_meta.shape)
for img_idx in range(mrcnn_config.BATCH_SIZE):
    print(img_meta[img_idx])
    image_id = img_meta[img_idx,0]
    image = dataset.load_image(image_id)
    timg  = train_batch_x[0][img_idx]
    print(' image from train_batch_x :', timg.shape, timg.dtype, np.min(timg), np.max(timg))
    print(' image from dataset load  :', image.shape, image.dtype, np.min(image), np.max(image))
    ## Display image, and mean-subtracted image
    visualize.display_image_bw(image)
#     visualize.display_images([image, train_batch_x[0][img_idx]], cols = 2, width = 18)
    
    ## display masks and bounding boxes
    mask, class_ids = dataset.load_mask(image_id)
    print('class_ids:', class_ids)
    bbox = utils.extract_bboxes(mask)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)
    visualize.display_instances_with_mask(image, bbox, mask, class_ids, dataset.class_names, figsize =(8,8)) 
    

    ## display ground truth heatmaps
    gt_class_ids = np.unique(train_batch_x[5][img_idx,:,:,4]).astype(int).tolist()
    print('Image : {}  GT ClassIds: {}'.format(img_idx, gt_class_ids))
#     visualize.plot_2d_heatmap_no_bboxes(train_batch_x[4],img_idx, columns = 4, class_names=class_names)    
    visualize.plot_2d_heatmap_no_bboxes(train_batch_x[4], img_idx,class_ids = gt_class_ids, columns = 4, class_names=class_names)
    
    
    ## display predicted heatmaps
    pr_class_ids = np.unique(train_batch_x[3][img_idx,:,:,4]).astype(int).tolist()
    print('Image : {}  PR ClassIds: {}'.format(img_idx, pr_class_ids))
#     visualize.plot_2d_heatmap_no_bboxes(train_batch_x[2],img_idx, columns = 4, class_names=class_names)
    visualize.plot_2d_heatmap_no_bboxes(train_batch_x[2], img_idx,class_ids = gt_class_ids, columns = 4, class_names=class_names)
        

In [ ]:
from mrcnn.utils import unresize_image
from   matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import skimage.util
import skimage.io
paths.display()


In [ ]:
# ds = os.path.join(paths.COCO_DATASET_PATH,'val2014/COCO_val2014_000000017031.jpg')
ds = dataset.image_info[210]['path']
print(ds)
im = skimage.io.imread(ds)
print(im.shape, im.dtype, np.min(im), np.max(im))
# im = skimage.io.imread(ds)
# im = Image.open(ds)

In [ ]:
print('image    : ', image.shape, image.dtype)
image_bw = Image.fromarray(image).convert(mode='L')
# print('image_bw : ', image_bw.shape, image_bw.dtype)

molded_image, window, scale, padding = utils.resize_image(
    image,
    min_dim=mrcnn_config.IMAGE_MIN_DIM,
    max_dim=mrcnn_config.IMAGE_MAX_DIM,
    padding=mrcnn_config.IMAGE_PADDING)
print('molded_image   : ', molded_image.shape, molded_image.dtype)
print(' image meta    :', train_batch_x[1][0])

unresized_image = unresize_image(molded_image,train_batch_x[1][0])
print('unresized_image : ', unresized_image.shape, unmolded_image.dtype)

unresized_image_bw = np.asarray(Image.fromarray(unresized_image).convert(mode='L'))
print('unresized_image_bw: ',unresized_image_bw.shape, unresized_image_bw.dtype)


unmolded_image = utils.unmold_image(molded_image, mrcnn_config)
print('unmolded_image : ', unmolded_image.shape, unmolded_image.dtype)

unmolded_image_bw = np.asarray(Image.fromarray(unmolded_image).convert(mode='L'))
print('unmolded_image_bw : ', unmolded_image_bw.shape, unmolded_image_bw.dtype)


unmolded_heatmap = unresize_image(train_batch_x[2][0,:,:,24],train_batch_x[1][0], upscale = mrcnn_config.HEATMAP_SCALE_FACTOR)
print('unmolded_heatmap : ', unmolded_heatmap.shape, unmolded_heatmap.dtype)


print(train_batch_x[2][0,:,:,24].dtype)




In [ ]:
import matplotlib.pyplot as plt
print('Orig image shape: ', image.shape)
print('Image window is : ', window)
print('Scale is        : ', scale)
print(train_batch_x[1][0,:8])
print('Padding is :', padding)
fig = plt.figure(frameon=False, figsize=(10,10))
im1 = plt.imshow(molded_image)



fig = plt.figure(frameon=False, figsize=(10,10))
im1 = plt.imshow(unresized_image)
fig = plt.figure(frameon=False, figsize=(10,10))
im1 = plt.imshow(unresized_image_bw, cmap=plt.cm.gray)

fig = plt.figure(frameon=False, figsize=(10,10))
im1 = plt.imshow(unmolded_image)
fig = plt.figure(frameon=False, figsize=(10,10))
im1 = plt.imshow(unmolded_image_bw, cmap=plt.cm.gray)
# fig = plt.figure(frameon=False, figsize=(10,10))
# im1 = plt.imshow(unmolded_image_bw)

# fig = plt.figure(frameon=False, figsize=(10,10))
# im1 = plt.imshow(unmolded_heatmap,cmap = cm.YlOrRd)


# fig = plt.figure(frameon=False, figsize=(10,10))
# im1 = plt.imshow(unmolded_image , cmap=plt.cm.gray)
# im1 = plt.imshow(unmolded_heatmap, alpha = 0.6,cmap=cm.YlOrRd)  


# fig = plt.figure(frameon=False, figsize=(10,10))
# im1 = plt.imshow(image_bw , cmap=plt.cm.gray)
# im1 = plt.imshow(unmolded_heatmap, alpha = 0.6,cmap=cm.YlOrRd)  
plt.show()

In [ ]:
from mrcnn.visualize import display_heatmaps, display_heatmaps_fcn, display_heatmaps_mrcnn
# visualize.display_image_bw(image)
display_heatmaps(train_batch_x, 0, hm = 'pr', config = mrcnn_config, class_ids = [0,1,2,3,4,5,24], class_names = dataset.class_names)
# display_heatmaps(train_batch_x, 0, hm = 'gt', config = mrcnn_config, class_ids = [0,1,2,3,4,5,24], class_names = dataset.class_names)

##  Build FCN Model 

In [ ]:
from mrcnn.utils import Paths
paths = Paths(fcn_training_folder='train_fcn8_coco')
paths.display()
##------------------------------------------------------------------------------------
## Build configuration for FCN model
##------------------------------------------------------------------------------------
fcn_config = CocoConfig()
# fcn_config.IMAGE_MAX_DIM        = 600
# fcn_config.IMAGE_MIN_DIM        = 480      
# mrcnn_config.COCO_DATASET_PATH  = COCO_DATASET_PATH 
# mrcnn_config.COCO_MODEL_PATH    = COCO_MODEL_PATH   
# mrcnn_config.RESNET_MODEL_PATH  = RESNET_MODEL_PATH 
fcn_config.NAME                 = 'fcn'              
fcn_config.TRAINING_PATH        = paths.FCN_TRAINING_PATH
fcn_config.VGG16_MODEL_PATH     = paths.FCN_VGG16_MODEL_PATH
fcn_config.FCN_INPUT_SHAPE      = mrcnn_config.IMAGE_SHAPE[0:2] // mrcnn_config.HEATMAP_SCALE_FACTOR 

fcn_config.BATCH_SIZE           = mrcnn_config.BATCH_SIZE                # Batch size is 2 (# GPUs * images/GPU).
fcn_config.IMAGES_PER_GPU       = mrcnn_config.BATCH_SIZE                  # Must match BATCH_SIZE
fcn_config.EPOCHS_TO_RUN        = 1
fcn_config.STEPS_PER_EPOCH      = 4
fcn_config.LAST_EPOCH_RAN       = 0

fcn_config.LEARNING_RATE        = 0.0001 

fcn_config.BATCH_MOMENTUM       = 0.9
fcn_config.WEIGHT_DECAY         = 2.0e-4
fcn_config.REDUCE_LR_FACTOR     = 0.5
fcn_config.REDUCE_LR_COOLDOWN   = 5
fcn_config.REDUCE_LR_PATIENCE   = 5
fcn_config.EARLY_STOP_PATIENCE  = 15
fcn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
fcn_config.MIN_LR               = 1.0e-10
 
fcn_config.VALIDATION_STEPS     = 5
fcn_config.REDUCE_LR_FACTOR     = 0.5
fcn_config.REDUCE_LR_COOLDOWN   = 50
fcn_config.REDUCE_LR_PATIENCE   = 33
fcn_config.EARLY_STOP_PATIENCE  = 50
fcn_config.EARLY_STOP_MIN_DELTA = 1.0e-6
fcn_config.MIN_LR               = 1.0e-10
fcn_config.NEW_LOG_FOLDER       = True  
fcn_config.OPTIMIZER            = 'ADAGRAD'
fcn_config.SYSOUT               = 'screen'

fcn_config.display()

#### Build Model

In [ ]:
##------------------------------------------------------------------------------------
## Build FCN Model in Training Mode
##------------------------------------------------------------------------------------
try :
    del fcn_model
    gc.collect()
except: 
    pass    
fcn_model = fcn_modellib.FCN(mode="training", arch = 'FCN8', config=fcn_config)

####  Display FCN model info

## Load Training Files and Display Images

In [ ]:
dataset_train, train_generator  = prep_coco_dataset(['train','val35k'], mrcnn_config, generator = True)
# dataset_val, val_generator      = prep_coco_dataset(['minival'], mrcnn_config, generator = True) 
class_names = dataset_train.class_names

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
imgmeta_idx = mrcnn_model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(mrcnn_config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    bbox = utils.extract_bboxes(mask)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
#     print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
    visualize.display_instances_with_mask(image, bbox, mask, class_ids, dataset_train.class_names, figsize =(8,8))  

#### Load a specific image using image_id

In [ ]:
## 62642 (persons),   68539 (trucks) 36466 (surfers)  75040 (boat and persons)
## 36466 surfers. 5498 basketbal players, 27711,30531
## 5498 lots of motorcylces & persons - 
## Persons: #26026, #7719, 111864, 58240,  
## 89243: Person, bicylce and traiffic lights
## 35347 - laptops, keyboards and cat
## items = [59199 , 102868]
## 101623 (cake and forks), 41423 (elephant & people)
## 33477 Table, bowl, cup, sandwich, knife
# train_batch_x, train_batch_y = next(train_generator)
# IMAGE_LIST = [75040] 
IMAGE_LIST = [89243]
train_batch_x, train_batch_y = test_batch_x, test_batch_y = data_gen_simulate(dataset_train, mrcnn_config, IMAGE_LIST)
imgmeta_idx = mrcnn_model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(mrcnn_config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    bbox = utils.extract_bboxes(mask)
    class_names = [str(dataset_train.class_names[class_id]) for class_id in class_ids]
    
    print(' Image meta  : ', img_meta[img_idx,:10])
    print(' Classes     : ', class_ids)
    print(" Image_id    : ", image_id, ' Reference: ', dataset_train.image_reference(image_id))
    print(' Class_ids.shape[0]:', class_ids.shape[0], 'bbox.shape[0]:',bbox.shape[0])       
    print(' Class Names : ', class_names)
    
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)   
    # Display image and instances
    visualize.display_instances_with_mask(image, bbox, mask, class_ids, dataset_train.class_names, figsize =(8,8))    